In [11]:
from __future__ import annotations
from pathlib import Path
from typing import Any

import pennylane as qml
from catalyst.passes import apply_pass
from mqt.core.plugins.catalyst import get_device

@qml.set_shots(1024)
@qml.qnode(get_device("lightning.qubit", wires=2))
def circuit() -> None:
    if True:
        qml.H(wires=0)
    else:
        qml.X(wires=0)
    qml.CNOT(wires=[0, 1])
    return


@qml.qjit(target="mlir", autograph=True, keep_intermediate=2)
def module() -> Any: 
    return circuit()

# Trigger JIT compilation and MLIR generation
module.mlir_opt

# Find where MLIR files are generated (relative to cwd where pytest is run)
# Catalyst generates MLIR files in the current working directory
mlir_dir = Path.cwd()

# Read the intermediate MLIR files
catalyst_mlir = mlir_dir / "0_catalyst_module.mlir"

if not catalyst_mlir.exists():
    available_files = list(mlir_dir.glob("*.mlir"))
    msg = f"Expected MLIR files not found in {mlir_dir}.\nAvailable files: {[f.name for f in available_files]}"
    raise FileNotFoundError(msg)

with Path(catalyst_mlir).open("r", encoding="utf-8") as f:
    mlir = f.read()

In [12]:
print(mlir)

module @module {
  func.func public @jit_module() attributes {llvm.emit_c_interface} {
    catalyst.launch_kernel @module_circuit::@circuit() : () -> ()
    return
  }
  module @module_circuit {
    module attributes {transform.with_named_sequence} {
      transform.named_sequence @__transform_main(%arg0: !transform.op<"builtin.module">) {
        transform.yield 
      }
    }
    func.func public @circuit() attributes {diff_method = "adjoint", llvm.linkage = #llvm.linkage<internal>, qnode} {
      %c = stablehlo.constant dense<1024> : tensor<i64>
      %extracted = tensor.extract %c[] : tensor<i64>
      quantum.device shots(%extracted) ["/Users/patrickhopf/Code/mqt/core-plugins-catalyst/.venv/lib/python3.12/site-packages/pennylane_lightning/liblightning_qubit_catalyst.dylib", "LightningSimulator", "{'mcmc': False, 'num_burnin': 0, 'kernel_name': None}"]
      %c_0 = stablehlo.constant dense<2> : tensor<i64>
      %0 = quantum.alloc( 2) : !quantum.reg
      %c_1 = stablehlo.constant 

### MLIR Structure Overview

```cpp
module @module                     // operation owning a region
 └─ region (module body)
     ├─ func.func @jit_module()    // operation owning a region
     │   └─ region (function body)
     │       └─ block               // sequence of operations
     └─ module @module_circuit     // operation owning a region
         └─ region (module body)
             └─ func.func @circuit()  // operation owning a region
                 └─ region (function body)
                     └─ block          // sequence of operations
                         %c = ...
                         %extracted = ...
                         quantum.device shots(%extracted) [...]
                         ...
                         return
```

<img src="/Users/patrickhopf/Code/mqt/core-plugins-catalyst/DefUseChains.svg"
     alt="MLIR Structure Overview"
     width="500"/>

<img src="/Users/patrickhopf/Code/mqt/core-plugins-catalyst/Use-list.svg"
     alt="MLIR Structure Overview"
     width="500"/>

### MLIR Program Representation

```cpp
module @module {                                          // Top-level MLIR module body// region
  func.func ... {                                         // Function operation// introduces a function with its own region
    ...                                                   // Contains Catalyst-specific and JIT-related IR
  }
  module @module_circuit {                                // Nested module// independent symbol table grouping circuit-related IR
    ...                                                   
    func.func public @circuit() attributes {...} {        // Public function symbol// no arguments and no return values

      // result = dialect.operation operand[syntax_sugar] : operand_type -> result_type
      %c = stablehlo.constant dense<1024> : tensor<i64> ->  tensor<i64>
      %extracted = tensor.extract %c[] : tensor<i64> -> i64

      // dialect.operation named_operand(operand)
      quantum.device shots(%extracted) [...]              // Side-effecting operation configuring a quantum device
      ...                                                 

      %0 = quantum.alloc( 2) : !quantum.reg               // Allocates a quantum register of size 2, producing a new SSA value

      %c_1 = stablehlo.constant dense<0> : tensor<i64>    // Constant tensor encoding index 0
      %extracted_2 = tensor.extract %c_1[] : tensor<i64>  // Extracts the scalar index value
      %1 = quantum.extract %0[%extracted_2]               // Extracts an element from the register
           : !quantum.reg -> !quantum.bit                 // Type conversion from register to single-bit handle

      // result = dialect.operation string_attribute(syntax_sugar) operand : operand_type
      %out_qubits = quantum.custom "Hadamard"() %1 : !quantum.bit

      %c_3 = stablehlo.constant dense<1> : tensor<i64>    
      %extracted_4 = tensor.extract %c_3[] : tensor<i64>  
      %2 = quantum.extract %0[%extracted_4] : !quantum.reg -> !quantum.bit

      %out_qubits_5:2 = quantum.custom "CNOT"() %out_qubits, %2 : !quantum.bit, !quantum.bit

      %extracted_6 = tensor.extract %c_1[] : tensor<i64>     // Re-extracts index 0 as an SSA value
      %3 = quantum.insert %0[%extracted_6], %out_qubits_5#0  // Inserts an updated element into the register
           : !quantum.reg, !quantum.bit

      %extracted_7 = tensor.extract %c_3[] : tensor<i64>
      %4 = quantum.insert %3[%extracted_7], %out_qubits_5#1 : !quantum.reg, !quantum.bit
      
      quantum.dealloc %4 : !quantum.reg                   // Explicit deallocation of the quantum register
      quantum.device_release                              // Releases the previously configured device
      return                                              // Function terminator (required even with no return values)
    }
  }
  ...                                                     // Other modules or operations may follow
}
```

### tablegen example

```cpp
def QuantumDeviceOp : Quantum_Op<"quantum.device", [SideEffecting]> {
  // This defines a new MLIR operation named `quantum.device` in the Quantum dialect.
  // `Quantum_Op` is a TableGen class that provides common behavior for quantum operations.
  // The first parameter "quantum.device" specifies the operation name as it appears in MLIR text.
  // The SideEffecting trait indicates that this operation has side effects (e.g., it modifies device state).
  
  let arguments = (ins
      I64:$shots            // Defines an operand for this operation.
                            // - Type: I64 (64-bit integer)
                            // - Name: $shots (used internally in TableGen and in assemblyFormat)
                            // - Represents the number of shots to execute on the quantum device
                            // - This will appear in MLIR text as `shots(%SSA_VALUE)` where %SSA_VALUE
                            //   is an SSA value of type i64 provided at runtime
  );

  let attributes = [
      StrAttr:$backend,      // String attribute representing the backend library or simulator
                             // Example: "/path/to/liblightning_qubit_catalyst.dylib"
                             // Stored in the operation's attribute dictionary internally
      StrAttr:$simulator,    // String attribute representing the simulator name
                             // Example: "LightningSimulator"
      DictionaryAttr:$config // Dictionary attribute storing additional configuration
                             // Example: {"mcmc": False, "num_burnin": 0, "kernel_name": None}
                             // Compile-time constant; cannot be SSA value
  ];

  let results = (outs);      // Defines the result types of the operation.
                             // Empty because `quantum.device` is side-effecting only
                             // and does not produce any SSA results.

  let assemblyFormat = "shots($shots) [$backend, $simulator, $config]";
                             // Defines how the operation is printed and parsed in MLIR text.
                             // - `$shots` refers to the operand defined in `let arguments`
                             // - `$backend`, `$simulator`, `$config` refer to attributes
                             // - Ensures MLIR can parse
                             //   `quantum.device shots(%extracted) ["/path/lib", "Sim", {...}]`
                             //   into the correct operand and attribute slots internally
                             // - Parentheses and brackets in assemblyFormat are purely for syntax; 
                             //   internally operands and attributes are stored separately
}
```

### Let's perform a roundtrip conversion

In [13]:
from __future__ import annotations
from pathlib import Path
from typing import Any

import pennylane as qml
from catalyst.passes import apply_pass
from mqt.core.plugins.catalyst import get_device

@apply_pass("mqt.mqtopt-to-catalystquantum") # <--- new
@apply_pass("mqt.catalystquantum-to-mqtopt") # <--- new
@qml.set_shots(1024)
@qml.qnode(get_device("lightning.qubit", wires=2))
def circuit() -> None:
    qml.H(wires=0)
    qml.CNOT(wires=[0, 1])
    if True:
        qml.RX(0.5, wires=0)
    return


custom_pipeline = [                                                         # <--- new
    # For demonstration we only want to use the two custom passes
    ("to-mqtopt", ["builtin.module(catalystquantum-to-mqtopt)"]),
    ("to-catalystquantum", ["builtin.module(mqtopt-to-catalystquantum)"]),
]

@qml.qjit(target="mlir", autograph=True, keep_intermediate=2, pipelines=custom_pipeline)
def module() -> Any: 
    return circuit()

# Trigger JIT compilation and MLIR generation
module.mlir_opt

# Find where MLIR files are generated (relative to cwd where pytest is run)
# Catalyst generates MLIR files in the current working directory
mlir_dir = Path.cwd()

# Read the intermediate MLIR files
catalyst_mlir = mlir_dir / "0_catalyst_module.mlir"
mlir_to_mqtopt = mlir_dir / "1_CatalystQuantumToMQTOpt.mlir"    # <--- new
mlir_to_catalyst = mlir_dir / "4_MQTOptToCatalystQuantum.mlir"  # <--- new

if not catalyst_mlir.exists() or not mlir_to_mqtopt.exists() or not mlir_to_catalyst.exists():
    available_files = list(mlir_dir.glob("*.mlir"))
    msg = f"Expected MLIR files not found in {mlir_dir}.\nAvailable files: {[f.name for f in available_files]}"
    raise FileNotFoundError(msg)

with Path(catalyst_mlir).open("r", encoding="utf-8") as f:
    mlir_before_conversion = f.read()
with Path(mlir_to_mqtopt).open("r", encoding="utf-8") as f:     # <--- new
    mlir_after_conversion = f.read()
with Path(mlir_to_catalyst).open("r", encoding="utf-8") as f:   # <--- new
    mlir_after_roundtrip = f.read()

### MLIR after conversion to MQTOpt

In [14]:
print(mlir_after_conversion)

module @module_circuit {
  module attributes {transform.with_named_sequence} {
    transform.named_sequence @__transform_main(%arg0: !transform.op<"builtin.module">) {
      %0 = transform.apply_registered_pass "catalystquantum-to-mqtopt" to %arg0 : (!transform.op<"builtin.module">) -> !transform.op<"builtin.module">
      %1 = transform.apply_registered_pass "mqtopt-to-catalystquantum" to %0 : (!transform.op<"builtin.module">) -> !transform.op<"builtin.module">
      transform.yield 
    }
  }
  func.func public @circuit() attributes {diff_method = "adjoint", llvm.linkage = #llvm.linkage<internal>, qnode} {
    %c = stablehlo.constant dense<1024> : tensor<i64>
    %extracted = tensor.extract %c[] : tensor<i64>
    quantum.device shots(%extracted) ["/Users/patrickhopf/Code/mqt/core-plugins-catalyst/.venv/lib/python3.12/site-packages/pennylane_lightning/liblightning_qubit_catalyst.dylib", "LightningSimulator", "{'mcmc': False, 'num_burnin': 0, 'kernel_name': None}"]
    %c_0 = stablehlo

### Differneces between CatalystQuantum and MQTOpt dialects

Now lets look at the same circuit but with `qml.RZ(0.5, wires=0)` instead of `qml.Hadamard(wires=0)`

```cpp
  func.func public @circuit() attributes {...} {
    ...
    %alloc = memref.alloc() : memref<2x!mqtopt.Qubit>

    %c_1 = stablehlo.constant dense<0> : tensor<i64>
    %extracted_2 = tensor.extract %c_1[] : tensor<i64>
    %0 = arith.index_cast %extracted_2 : i64 to index
    %1 = memref.load %alloc[%0] : memref<2x!mqtopt.Qubit>

    %cst = stablehlo.constant dense<5.000000e-01> : tensor<f64>
    %extracted_3 = tensor.extract %cst[] : tensor<f64>
    // static and mask are used to handle gate parameters that can be either static (compile-time constants) or dynamic (runtime values represented as SSA values)
    %out_qubits = mqtopt.rz(%extracted_3 static [] mask [false]) %1 : !mqtopt.Qubit

    %c_4 = stablehlo.constant dense<1> : tensor<i64>
    %extracted_5 = tensor.extract %c_4[] : tensor<i64>
    %2 = arith.index_cast %extracted_5 : i64 to index
    %3 = memref.load %alloc[%2] : memref<2x!mqtopt.Qubit>

    %out_qubits_6, %pos_ctrl_out_qubits = mqtopt.x(static [] mask []) %3 ctrl %out_qubits : !mqtopt.Qubit ctrl !mqtopt.Qubit

    %extracted_7 = tensor.extract %c_1[] : tensor<i64>
    %4 = arith.index_cast %extracted_7 : i64 to index
    memref.store %pos_ctrl_out_qubits, %alloc[%4] : memref<2x!mqtopt.Qubit>
    %extracted_8 = tensor.extract %c_4[] : tensor<i64>
    %5 = arith.index_cast %extracted_8 : i64 to index
    memref.store %out_qubits_6, %alloc[%5] : memref<2x!mqtopt.Qubit>
    
    memref.dealloc %alloc : memref<2x!mqtopt.Qubit>
    ...
    return
  }
```

### Lowering to the Quantum Intermediate Representation (QIR) - [Base Profile](https://github.com/qir-alliance/qir-spec/blob/main/specification/profiles/Base_Profile.md#quantum-instruction-set)

```cpp
// === QIR Types ===
// QIR uses opaque types for quantum resources
%Qubit = type opaque    // Represents a single qubit (handle managed by the runtime/QPU)
%Array = type opaque    // Represents an array of qubits (handle managed by the runtime/QPU)

// === QIR Runtime / Quantum Instruction Declarations ===
// These functions must be implemented by a QIR-compliant runtime / quantum device backend.

declare void @__quantum__rt__initialize(i8* nocapture)
// Initialize the quantum runtime (set up device, backend, configuration)

declare void @__quantum__rt__finalize()
// Finalize the quantum runtime, release resources

declare i8* @__quantum__rt__qubit_allocate_array(i64)
// Allocate a qubit register of given size
// Returns a pointer to a Qubit array (runtime-managed memory)

declare void @__quantum__rt__qubit_release_array(i8* nocapture)
// Release a previously allocated qubit register
// Must be called after all gates/measurements are done

declare i8* @__quantum__rt__array_get_element_ptr_1d(i8*, i64)
// Obtain a pointer to a specific qubit in a Qubit array
// Equivalent to indexing into a quantum register

declare void @__quantum__rt__device_release()
// Release all device-level resources
// Called after the circuit execution is complete

// QIR Quantum Instruction Set (QIS) functions
declare void @__quantum__qis__h(%Qubit*)
// Hadamard gate on a single qubit
// This operation must be implemented by the QPU or simulator

declare void @__quantum__qis__cnot(%Qubit*, %Qubit*)
// CNOT gate: first qubit is control, second is target
// This operation must be implemented by the QPU or simulator

// === Global configuration strings (passed to runtime) ===
@.cfg_str = internal constant [55 x i8] c"{'mcmc': False, 'num_burnin': 0, 'kernel_name': None}\00"
@.sim_name = internal constant [20 x i8] c"LightningSimulator\00"
@.lib_path = internal constant [140 x i8] c"/Users/.../liblightning_qubit_catalyst.dylib\00"

// === Circuit Implementation ===
define void @circuit() {
entry:
  // --------------------------
  // Initialize device / runtime
  // --------------------------
  %cfg_ptr = getelementptr inbounds [55 x i8], [55 x i8]* @.cfg_str, i64 0, i64 0
  %sim_ptr = getelementptr inbounds [20 x i8], [20 x i8]* @.sim_name, i64 0, i64 0
  %lib_ptr = getelementptr inbounds [140 x i8], [140 x i8]* @.lib_path, i64 0, i64 0
  call void @__quantum__rt__initialize(%lib_ptr)
  // QPU / simulator is now ready

  // --------------------------
  // Allocate qubit register
  // --------------------------
  %arr = call i8* @__quantum__rt__qubit_allocate_array(i64 2)
  // Runtime returns a handle to 2 qubits
  // All quantum operations below operate on pointers obtained from this array

  // --------------------------
  // Access individual qubits
  // --------------------------
  %elem0_addr = call i8* @__quantum__rt__array_get_element_ptr_1d(i8* %arr, i64 0)
  %q0 = bitcast i8* %elem0_addr to %Qubit*
  // Qubit 0 handle

  %elem1_addr = call i8* @__quantum__rt__array_get_element_ptr_1d(i8* %arr, i64 1)
  %q1 = bitcast i8* %elem1_addr to %Qubit*
  // Qubit 1 handle

  // --------------------------
  // Apply quantum operations (must be implemented by QPU)
  // --------------------------
  call void @__quantum__qis__h(%Qubit* %q0)
  // Apply Hadamard to qubit 0

  call void @__quantum__qis__cnot(%Qubit* %q0, %Qubit* %q1)
  // Apply CNOT: q0 = control, q1 = target

  // --------------------------
  // Cleanup
  // --------------------------
  call void @__quantum__rt__qubit_release_array(i8* %arr)
  // Release all allocated qubits

  call void @__quantum__rt__device_release()
  // Release backend / simulator resources

  ret void
}

// === Setup / Teardown ===
define void @setup() {
entry:
  call void @__quantum__rt__initialize(i8* null)
  // Optional: initialize runtime before any circuit execution
  ret void
}

define void @teardown() {
entry:
  call void @__quantum__rt__finalize()
  // Optional: finalize runtime after all circuits
  ret void
}
```